In [8]:
import pickle
import numpy as np

In [9]:
with open('embeddings_all.pkl', 'rb') as file:
    data = pickle.load(file)

x = data["outputs"].cpu().numpy()
y = data["labels"].cpu().numpy()
print(y.shape)

(49984,)


In [10]:
# normalize embeddings
x = (x - np.min(x)) / (np.max(x) - np.min(x))

In [11]:
embeddings = np.array(x)

In [12]:
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

def cluster_classes(embeddings, labels, n_clusters=10, classes_per_cluster=10):
    # Initialize clusters
    clusters = {i: [] for i in range(n_clusters)}
    unassigned_labels = set(labels)

    # Calculate and store all pairwise distances
    distances = euclidean_distances(embeddings, embeddings)

    # Precompute the minimum distance to each unassigned label from each cluster
    min_distances = {label: np.inf for label in unassigned_labels}

    for cluster_id in range(n_clusters):
        for _ in range(classes_per_cluster):
            if cluster_id == 0:
                # For the first cluster, choose classes randomly
                chosen_label = np.random.choice(list(unassigned_labels))
            else:
                # For subsequent clusters, choose based on minimum cumulative distance
                chosen_label = min(unassigned_labels, key=lambda label: min_distances[label])

            # Assign chosen label to current cluster and remove from unassigned labels
            clusters[cluster_id].append(chosen_label)
            unassigned_labels.remove(chosen_label)

            # Update minimum distances for the remaining unassigned labels
            for label in unassigned_labels:
                min_distances[label] = min(min_distances[label], distances[chosen_label, label])

    return clusters
# Example usage
# embeddings = np.array([...])  # Replace with your embeddings array
# labels = np.array([...])      # Replace with your labels array
# result = cluster_classes(embeddings, labels)
# print(result)

In [13]:
result = cluster_classes(embeddings, y)

In [15]:
print(result)

{0: [69, 12, 54, 76, 97, 64, 9, 85, 73, 47], 1: [74, 45, 6, 4, 26, 36, 17, 81, 82, 33], 2: [48, 98, 77, 16, 67, 80, 95, 0, 99, 83], 3: [51, 75, 44, 79, 13, 41, 28, 24, 63, 34], 4: [35, 56, 37, 84, 2, 15, 87, 10, 88, 89], 5: [61, 42, 43, 30, 70, 62, 91, 68, 19, 55], 6: [53, 27, 7, 60, 22, 86, 20, 8, 38, 49], 7: [92, 93, 94, 72, 40, 58, 1, 96, 21, 29], 8: [52, 18, 5, 11, 59, 71, 14, 46, 65, 57], 9: [90, 25, 78, 50, 66, 23, 31, 32, 3, 39]}


In [17]:
reverse = {}

for key in result.keys():
    for i in range(len(result[key])):
        reverse[result[key][i]] = key

print(reverse)


{69: 0, 12: 0, 54: 0, 76: 0, 97: 0, 64: 0, 9: 0, 85: 0, 73: 0, 47: 0, 74: 1, 45: 1, 6: 1, 4: 1, 26: 1, 36: 1, 17: 1, 81: 1, 82: 1, 33: 1, 48: 2, 98: 2, 77: 2, 16: 2, 67: 2, 80: 2, 95: 2, 0: 2, 99: 2, 83: 2, 51: 3, 75: 3, 44: 3, 79: 3, 13: 3, 41: 3, 28: 3, 24: 3, 63: 3, 34: 3, 35: 4, 56: 4, 37: 4, 84: 4, 2: 4, 15: 4, 87: 4, 10: 4, 88: 4, 89: 4, 61: 5, 42: 5, 43: 5, 30: 5, 70: 5, 62: 5, 91: 5, 68: 5, 19: 5, 55: 5, 53: 6, 27: 6, 7: 6, 60: 6, 22: 6, 86: 6, 20: 6, 8: 6, 38: 6, 49: 6, 92: 7, 93: 7, 94: 7, 72: 7, 40: 7, 58: 7, 1: 7, 96: 7, 21: 7, 29: 7, 52: 8, 18: 8, 5: 8, 11: 8, 59: 8, 71: 8, 14: 8, 46: 8, 65: 8, 57: 8, 90: 9, 25: 9, 78: 9, 50: 9, 66: 9, 23: 9, 31: 9, 32: 9, 3: 9, 39: 9}


In [ ]:
# for each embedding fint the closest centroid

a = np.zeros((10,100))


for i, x in enumerate(embeddings):
    distances = np.linalg.norm(result - x, axis=1)
    first_index = np.argsort(distances)[0]    
    a[reverse[first_index]][y[i]] += 1
